In [230]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
import tensorflow
from sklearn import model_selection
from sklearn import metrics
import matplotlib.pyplot as plt

In [231]:
data = pd.read_csv('data.csv')

In [232]:
data.drop(columns=['Unnamed: 0'], inplace = True)

In [233]:
y = data.iloc[:, -1]

In [234]:
X = data.iloc[:, :data.shape[1] - 1]

In [235]:
cnn_par_f = {
    'window': [3, 7, 11],
    'dropout': [0.1, 0.2, 0.5]
}
cnn_par_raw= {
    'window': [3, 11, 15, 19],
    'dropout': [0.1, 0.2, 0.5]
}

In [236]:
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import GlobalAveragePooling1D
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier

In [237]:
from sklearn import preprocessing

In [238]:
filter_cnn = 64
num_classes = len(np.unique(y))
epochs = 25
K=10

In [239]:
pool = 3
classes = np.unique(y)
num_classes = len(classes)

In [240]:
scoring = {'f1_macro' : 'f1_macro', 'accuracy' : 'accuracy'}
for c in classes:
    def f1_class(y_true, y_pred, classes, c=c):
        return metrics.f1_score(y_true, y_pred, average=None)[c-1]
    scoring_name = 'f1_class_%d' % c
    scoring[scoring_name] = metrics.make_scorer(f1_class, classes = classes)

## Bez transformacija podataka

In [241]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.3, stratify = y,random_state = 1996)

In [242]:
scaler_train = preprocessing.StandardScaler()
scaler_train.fit(X_train)
X_train = scaler_train.transform(X_train)
X_test = scaler_train.transform(X_test)

In [243]:
X_train = np.array(X_train)
X_test = np.array(X_test)

In [244]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [245]:
input_shape = (X_train.shape[1], 1)

In [246]:
def mreza(input_shape, num_classes,window, dropout):
    model = Sequential()
    model.add(Conv1D(filter_cnn, window, activation='relu', input_shape=input_shape))
    model.add(Conv1D(filter_cnn, window, activation='relu'))
    model.add(MaxPooling1D(pool))
    model.add(Conv1D(filter_cnn * 2, window, activation='relu'))
    model.add(Conv1D(filter_cnn * 2, window, activation='relu'))
    model.add(GlobalAveragePooling1D())
    model.add(Dropout(dropout))
    model.add(Dense(number_of_classes, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [247]:
from keras.wrappers.scikit_learn  import KerasClassifier
def get_estimator(window=3,dropout=0.5):
    return KerasClassifier(build_fn=mreza,input_shape=input_shape, num_classes=num_classes,window=window, dropout=dropout,epochs=epochs)

In [248]:
estimator = get_estimator()

In [249]:
grid_cnn = model_selection.GridSearchCV(estimator, param_grid=cnn_par, scoring=scoring, cv=10, refit='accuracy', return_train_score=True)

In [ ]:
grid_cnn.fit(X_train, y_train)

Epoch 1/25
2656/7245 [=========>....................] - ETA: 9s - loss: 1.3107 - accuracy: 0.3611

In [ ]:
grid_cnn.best_score_

In [ ]:
grid_cnn.best_estimator

In [ ]:
grid_cnn.best_params_

In [ ]:
best = grid_cnn.best_params_

In [ ]:
model = Sequential()
model.add(Conv1D(filter_cnn, window, activation='relu', input_shape=input_shape))
model.add(Conv1D(filter_cnn, window, activation='relu'))
model.add(MaxPooling1D(pool))
model.add(Conv1D(filter_cnn * 2, window, activation='relu'))
model.add(Conv1D(filter_cnn * 2, window, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(dropout))
model.add(Dense(number_of_classes, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=25 ,window = best['window'],dropout = best['dropout'], batch_size= 32, verbose = 0)

In [ ]:
plt.title('Treniranje mreže - funkcija gubitka')
plt.xlabel('Epoha')
plt.ylabel('Funkcija gubitka')
plt.plot(history.epoch, history.history['loss'])

In [ ]:
plt.title('Treniranje mreže - tačnost modela')
plt.xlabel('Epoha')
plt.ylabel('Tačnost')
plt.plot(history.epoch, history.history['accuracy'])

In [ ]:
model.save('models/cnn_raw.hdf5')

In [ ]:
#from keras.models import load_model
#model_revived = load_model('models/boston_housing.hdf5')

## Furijeova transformacija

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.3, stratify = y,random_state = 1996)

In [ ]:
scaler_train = preprocessing.StandardScaler()
scaler_train.fit(X_train)
X_train = scaler_train.transform(X_train)
X_test = scaler_train.transform(X_test)

In [192]:
X_f_train = np.fft.rfft(X_train, axis=1).astype(float)
X_f_test = np.fft.rfft(X_test, axis=1).astype(float)

c:\users\ana\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: ComplexWarning: Casting complex values to real discards the imaginary part
  """Entry point for launching an IPython kernel.
c:\users\ana\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: ComplexWarning: Casting complex values to real discards the imaginary part
  


In [ ]:
X_train.shape

In [ ]:
X_f_train.shape

In [ ]:
X_f_train = X_f_train.reshape(X_f_train.shape[0], X_f_train.shape[1], 1)
X_f_test = X_f_test.reshape(X_f_test.shape[0], X_f_test.shape[1], 1)

In [ ]:
input_shape = (X_f_train.shape[1], 1)

In [ ]:
def mreza(input_shape, num_classes,window, dropout):
    model = Sequential()
    model.add(Conv1D(filter_cnn, window, activation='relu', input_shape=input_shape))
    model.add(Conv1D(filter_cnn, window, activation='relu'))
    model.add(MaxPooling1D(pool))
    model.add(Conv1D(filter_cnn * 2, window, activation='relu'))
    model.add(Conv1D(filter_cnn * 2, window, activation='relu'))
    model.add(GlobalAveragePooling1D())
    model.add(Dropout(dropout))
    model.add(Dense(number_of_classes, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
from keras.wrappers.scikit_learn  import KerasClassifier
def get_estimator(window=3,dropout=0.5):
    return KerasClassifier(build_fn=mreza,input_shape=input_shape, num_classes=num_classes,window=window, dropout=dropout,epochs=epochs)

In [ ]:
estimator = get_estimator()

In [ ]:
grid_cnn_f = model_selection.GridSearchCV(estimator, param_grid=cnn_par_f, scoring=scoring, cv=10, refit='accuracy', return_train_score=True)

In [ ]:
grid_cnn_f.fit(X_f_train, y_train)